In [4]:
import pandas as pd
import numpy as np
import glob
import librosa
import wave
import struct
import math
import os
from scipy import fromstring, int16

In [23]:
out_folder = "../data/THINKLET/cut_wav/VID_20201117_141446/"

# 一応既に同じ名前のディレクトリがないか確認。
file = os.path.exists(out_folder)

if file == False:
    #保存先のディレクトリの作成
    os.mkdir(out_folder)

#filenameに読み込むファイル、timeにカットする間隔
def cut_wav(filename,time):  
    # timeの単位は[sec]

    # ファイルを読み出し
    wavf = filename + '.wav'
    wr = wave.open(wavf, 'r')

    # waveファイルが持つ性質を取得
    ch = wr.getnchannels()
    width = wr.getsampwidth()
    fr = wr.getframerate()
    fn = wr.getnframes()
    total_time = 1.0 * fn / fr 
    integer = math.floor(total_time*100) # 小数点以下切り捨て
    t = int(time*100)  # 秒数[sec]
    frames = int(ch * fr * t /100)
    num_cut = int(integer//t)
    # waveの実データを取得し、数値化
    data = wr.readframes(wr.getnframes())
    wr.close()
    X = np.frombuffer(data, dtype=int16)

    for i in range(num_cut):
        #print(i)
        # 出力データを生成
        outf = out_folder + str(i) + '.wav' 
        start_cut = int(i*frames)
        end_cut = int(i*frames + frames)
        #print(start_cut)
        #print(end_cut)
        Y = X[start_cut:end_cut]
        outd = struct.pack("h" * len(Y), *Y)

        # 書き出し
        ww = wave.open(outf, 'w')
        ww.setnchannels(ch)
        ww.setsampwidth(width)
        ww.setframerate(fr)
        ww.writeframes(outd)
        ww.close()

f_name = "../data/THINKLET/wav/VID_20201117_141446"
cut_time = 0.5
cut_wav(f_name,float(cut_time))

In [37]:
list_ceps = []
list_wav_names = []

# normalのwavを処理
folder_name = "VID_20201117_145508"
filelist = glob.glob('../data/THINKLET/cut_wav/' + folder_name + '/*.wav')  # ワイルドカードでnormal_*.wavのリストを作成

wav_dict = {}
for filename in filelist:
    y, sr = librosa.core.load(filename,sr=None)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20)
    
    # 複数のローリングウィンドウでそれぞれ20次元のMFCCを得られるので、その平均をとる。
    ceps = mfcc.mean(axis=1)
    
    # 辞書に追加
    wav_dict[int(filename.split("/")[-1].split(".")[0])] = ceps

# 配列に整形
wav_list = []
for i in range(1,max(wav_dict.keys())+1):
    wav_list.append(wav_dict[i])
    
wav_array = np.array(wav_list)

np.savez('../data/THINKLET/vec/' + folder_name, wav_array)